In [1]:
# # stdlib
# import os

# os.environ["ORCHESTRA_DEPLOYMENT_TYPE"] = "remote"
# os.environ["DEV_MODE"] = "True"
# os.environ["TEST_EXTERNAL_REGISTRY"] = "k3d-registry.localhost:5800"

In [2]:
import os

os.environ['ORCHESTRA_DEPLOYMENT_TYPE'] = 'remote'
os.environ['DEVSPACE_PROFILE'] = 'bigquery-scenario-tests'
os.environ['GITHUB_CI'] = 'false'
os.environ['SYFT_BASE_IMAGE_REGISTRY'] = 'k3d-registry.localhost:5800'
os.environ['DATASITE_CLUSTER_NAME'] = 'test-datasite-1'
os.environ['SERVER_URL'] = 'http://localhost'
os.environ['SERVER_PORT'] = '9081'
os.environ['TEST_EXTERNAL_REGISTRY'] = 'k3d-registry.localhost:5800'
os.environ['TEST_QUERY_LIMIT_SIZE'] = '500000'
os.environ['TRACING'] = 'False'
os.environ['NUM_TEST_USERS'] = '5'
os.environ['NUM_TEST_JOBS'] = '10'
os.environ['NUM_TEST_WORKERS'] = '2'

In [2]:
# stdlib
from os import environ as env

# third party
# run email server
from helpers import get_email_server

# syft absolute
import syft as sy

# Launch & login

In [3]:
# when in k8s these are the default values
ROOT_EMAIL = "admin@bigquery.org"
ROOT_PASSWORD = "bqpw"

In [4]:
# in case we are not in k8s we set them here for orchestra to use
env["DEFAULT_ROOT_EMAIL"] = ROOT_EMAIL
env["DEFAULT_ROOT_PASSWORD"] = ROOT_PASSWORD

In [5]:
server = sy.orchestra.launch(
    name="bigquery-high",
    dev_mode=True,
    server_side_type="high",
    reset=True,
    port="9081",
    n_consumers=1,  # How many workers to be spawned
    create_producer=True,  # Can produce more workers
)

In [6]:
email_server, smtp_server = get_email_server(reset=True)

> Starting SMTPTestServer on: 0.0.0.0:9025


In [8]:
root_client = sy.login(
    url="http://localhost:9081", email=ROOT_EMAIL, password=ROOT_PASSWORD
)

ConnectionError: HTTPConnectionPool(host='localhost', port=8080): Max retries exceeded with url: /api/v2/metadata (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x3057ff800>: Failed to establish a new connection: [Errno 61] Connection refused'))

# Create new admin client

In [ ]:
ADMIN_EMAIL, ADMIN_PW = "admin2@bigquery.org", "bqpw2"

In [ ]:
# create
root_client.register(
    name="second admin", email=ADMIN_EMAIL, password=ADMIN_PW, password_verify=ADMIN_PW
)
# update role
new_user_id = root_client.users.search(email=ADMIN_EMAIL)[0].id
root_client.users.update(uid=new_user_id, role="admin")

# We cannot delete the root client

In [ ]:
high_client = sy.login(
    url="http://localhost:9081", email=ADMIN_EMAIL, password=ADMIN_PW
)

In [ ]:
root_admin_id = root_client.users.search(email=ROOT_EMAIL)[0].id

In [ ]:
with sy.raises(sy.SyftException):
    high_client.users.delete(root_admin_id)

# Create ephemeral admin and delete it

In [ ]:
# register 2nd new admin (to delete)
user_email, user_pw = "admin3@bigquery.org", "bqpw3"
## create
root_client.register(
    name="x", email=user_email, password=user_pw, password_verify=user_pw
)
## update role
new_user_id2 = root_client.users.search(email=user_email)[0].id
root_client.users.update(uid=new_user_id, role="admin")

In [ ]:
root_client.users.delete(new_user_id2)

# Cleanup

In [ ]:
smtp_server.stop()

In [ ]:
server.land()